In [1]:
import pandas as pd
import numpy as np

In [3]:
hotel = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/bora/hotel_bangkok.csv').iloc[:,[0,1,3,4]]
hotel.columns = ['Hotel_Ref','Hotel_Name','Hotel_Address','N']
hotel["Hotel_Address"]= hotel["Hotel_Address"].replace("บ้านเลขที่ ","", regex=True)
hotel.head(20)

,Hotel_Ref,Hotel_Name,Hotel_Address,N
0,101/2563,โรงแรม สุทธิสารพ้อยส์,45 ซ.ยิ้มอุปถัมภ์ แขวงรัชดาภิเษก เขตดินแดง จ.ก...,42
1,136/2563,โรงแรม คณาบูทีค 2,"3,5,7,9,11 ซ.จรัญสนิทวงศ์ 57/2 แขวงบางบำหรุ เข...",24
2,152/2563,วัลย์ลดา เพลส,71/1 ซ.รามคำแหง 53 (จันทร์ศรีชวาลา) แขวงพลับพล...,50
3,214/2563,โรงแรม นิวสยาม พาเลซวิลล์ 2,56 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
4,215/2563,โรงแรม นิวสยาม พาเลซวิลล์ 3,58 ตรอกโรงไหม แขวงชนะสงคราม เขตพระนคร จ.กรุงเท...,48
5,6/2564,นิววังทอง,304 ถ.มหาชัย แขวงสำราญราษฎร์ เขตพระนคร จ.กรุงเ...,46
6,7/2564,39,6 ถ.บูรณศาสตร์ แขวงศาลเจ้าพ่อเสือ เขตพระนคร จ....,49
7,14/2564,นครพิงค์,9/1 ซ.วรพงษ์ แขวงบ้านพานถม เขตพระนคร จ.กรุงเทพ...,118
8,17/2564,พัฒนา(28),233 ถ.สามเสน แขวงวัดสามพระยา เขตพระนคร จ.กรุงเ...,79
9,32/2564,เค.ที,115/6 ถ.ประชาธิปไตย แขวงบางขุนพรหม เขตพระนคร จ...,105


## Method 1: Semantic search

In [8]:
trip_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/refs/heads/main/OTA/trip_com_bkk.csv')[['name','name href','address']].dropna()
trip_com['OTA'] = 'trip_com'

booking_com = pd.read_csv('https://raw.githubusercontent.com/Thanrada2024/illegal_accommodation/main/OTA/booking_com_bkk_with_addr.csv', engine='python')[['name href','name','address']].dropna()
booking_com['OTA'] = 'booking_com'

# รวมสอง DataFrame เข้าด้วยกัน
OTA_df = pd.concat([trip_com, booking_com], ignore_index=True)
OTA_df

,name,name href,address,OTA
0,นาซ่า กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"44 ถ. สุขุมวิท 71 แขวงสวนหลวง, เขตสวนหลวง, กรุ...",trip_com
1,โรงแรมอีสติน แกรนด์ พญาไท,https://th.trip.com/hotels/detail/?cityId=359&...,"18 ถ.พญาไท ทุ่งพญาไท, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
2,โรงแรมเอเชียกรุงเทพ,https://th.trip.com/hotels/detail/?cityId=359&...,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com
3,โรงแรมทวิน ทาวเวอร์,https://th.trip.com/hotels/detail/?cityId=359&...,"88 รองเมือง ปทุมวัน, ถ.พระราม 6, เขตปทุมวัน, ก...",trip_com
4,โรงแรมเดอะ เบอร์เคลีย์ ประตูน้ำ,https://th.trip.com/hotels/detail/?cityId=359&...,"559 ราชปรารภ, มักกะสัน ราชเทวี, เขตราชเทวี, กร...",trip_com
...,...,...,...,...
2660,บรรทัดทองโฮสเทล,https://www.booking.com/hotel/th/banthat-thong...,"1806/16-17 ถ.บรรทัดทอง รองเมือง ปทุมวัน, ปทุมว...",booking_com
2661,Vaeh Viean Silom,https://www.booking.com/hotel/th/vaeh-viean-ho...,"23 Pramot Alley, บางรัก, 10500 กรุงเทพมหานคร, ไทย",booking_com
2662,พาราดิโซบูติคสวีท,https://www.booking.com/hotel/th/paradiso-bout...,"1/11-12 สุขุมวิท 10, คลองเตย, 10110 กรุงเทพมหา...",booking_com
2663,Silom Space Hostel,https://www.booking.com/hotel/th/silom-space-h...,"8 Silom Road Soi 2, Bangrak, บางรัก, 10500 กรุ...",booking_com


In [10]:
# ---------- ตั้งค่า ----------
import re, unicodedata
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
# =========================
# 0) เตรียมตารางต้นทาง
# =========================
# trip_com: ต้องมีคอลัมน์ ['name', 'name href', 'address']
trip = OTA_df.rename(columns={
    "name": "Trip_Name",
    "name href": "Trip_URL",
    "address": "Trip_Address"
}).copy()

# hotel_list: ต้องมีคอลัมน์ชื่อ/ที่อยู่ตามตัวอย่างในภาพ
hotels = hotel.copy()
if "v-data-table__td 2" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 2": "Hotel_Name"})
if "v-data-table__td 4" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 4": "Hotel_Address"})
if "v-data-table__td" in hotels.columns:
    hotels["Hotel_Ref"] = hotels["v-data-table__td"]  # เลขอ้างอิงเอกสาร/ไอดี

for c in ["Trip_Name","Trip_Address"]:
    if c not in trip.columns: trip[c] = ""
trip[["Trip_Name","Trip_Address"]] = trip[["Trip_Name","Trip_Address"]].fillna("")

for c in ["Hotel_Name","Hotel_Address"]:
    if c not in hotels.columns: hotels[c] = ""
hotels[["Hotel_Name","Hotel_Address"]] = hotels[["Hotel_Name","Hotel_Address"]].fillna("")

# =========================
# 1) Clean: ชื่อ/ที่อยู่ + สร้าง "คีย์" name+address
# =========================
def normalize_name(s: str) -> str:
    """ตัดคำ 'โรงแรม/Hotel' และ 'กรุงเทพ/กรุงเทพฯ/กรุงเทพมหานคร' + ลบเว้นวรรค/สัญลักษณ์"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)  # ตัดคำ 'กรุงเทพ' ออกจาก 'ชื่อ'
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)  # ลบช่องว่างทั้งหมด
    return s

def normalize_addr(s: str) -> str:
    """คงโครงสร้างที่อยู่ไว้ (ไม่ลบช่องว่างทั้งหมด) แต่ normalize ให้เท่า ๆ กัน"""
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()

    # ทำให้คำหลักสอดคล้องกัน
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",              # จ.กรุงเทพมหานคร -> กรุงเทพมหานคร
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",        # กันกรณีมีช่องว่างเพี้ยน
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)

    # เก็บตัวอักษร/ตัวเลข/เว้นวรรคและเครื่องหมาย / , - บางส่วน
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

# apply clean
trip["Trip_Name_n"]    = trip["Trip_Name"].map(normalize_name)
trip["Trip_Address_n"] = trip["Trip_Address"].map(normalize_addr)
hotels["Hotel_Name_n"]    = hotels["Hotel_Name"].map(normalize_name)
hotels["Hotel_Address_n"] = hotels["Hotel_Address"].map(normalize_addr)

# คีย์ค้นหา = name + address (ตามที่ต้องการ)
trip["Trip_Key"]   = (trip["Trip_Name_n"]   + " " + trip["Trip_Address_n"]).str.strip()
hotels["Hotel_Key"] = (hotels["Hotel_Name_n"] + " " + hotels["Hotel_Address_n"]).str.strip()

# =========================
# 2) Embeddings
# =========================
MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# MODEL = "intfloat/multilingual-e5-large"

model = SentenceTransformer(MODEL)

# embeddings แยก (สำหรับรายงาน sim_name / sim_addr) + คีย์รวม (สำหรับค้นหา)
emb_trip_name  = model.encode(trip["Trip_Name_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_addr  = model.encode(trip["Trip_Address_n"].tolist(), convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_trip_key   = model.encode(trip["Trip_Key"].tolist(),      convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

emb_hotel_name = model.encode(hotels["Hotel_Name_n"].tolist(),   convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_addr = model.encode(hotels["Hotel_Address_n"].tolist(), convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)
emb_hotel_key  = model.encode(hotels["Hotel_Key"].tolist(),      convert_to_numpy=True, normalize_embeddings=True, batch_size=128, show_progress_bar=True)

# =========================
# 3) ค้นหาคู่โดยใช้ "คีย์รวม" แล้วคำนวณ sim_name/sim_addr แยก
# =========================
# ใช้ NearestNeighbors เพื่อสเกลกับข้อมูลใหญ่ และใช้ cosine distance
nn = NearestNeighbors(n_neighbors=1, metric="cosine")
nn.fit(emb_hotel_key)
dist, idx = nn.kneighbors(emb_trip_key, return_distance=True)
sim_key = 1.0 - dist.ravel()        # cosine similarity
best_idx = idx.ravel()

# similarity แยกชื่อ/ที่อยู่ กดดอทกับคู่ที่เลือก (embedding ถูก normalize แล้ว)
best_name = np.einsum("ij,ij->i", emb_trip_name, emb_hotel_name[best_idx])
best_addr = np.einsum("ij,ij->i", emb_trip_addr, emb_hotel_addr[best_idx])

# เกณฑ์ผ่านหลักจากคีย์รวม (name+address)
KEY_THRESH = 0.95   # ปรับตามการแจกแจงของข้อมูลคุณ
is_match = (sim_key >= KEY_THRESH)



# =========================
# 4) สร้างผลลัพธ์ โดยยึด trip_com เป็นหลัก
# =========================
matched_hotels = hotels.iloc[best_idx].reset_index(drop=True)

trip_match = trip.copy()
trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values,    np.nan)
trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
trip_match["Match_Hotel_Ref"]     = np.where(
    is_match,
    matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip))).values,
    np.nan
)

# similarity แยก + รวม
trip_match["sim_name"] = best_name
trip_match["sim_addr"] = best_addr
trip_match["sim_key"]  = sim_key     # ความคล้ายของคีย์ name+address
trip_match["is_match"] = is_match

# 4.1 เติม best-candidate ให้ครบทุกแถว
trip_match["Best_Hotel_Name"]    = matched_hotels["Hotel_Name"].values
trip_match["Best_Hotel_Address"] = matched_hotels["Hotel_Address"].values
trip_match["Best_Hotel_Ref"]     = matched_hotels.get(
    "Hotel_Ref", pd.Series([np.nan]*len(trip_match))
).values

# 4.2 คง Match_* สำหรับเฉพาะแถวที่ผ่านเกณฑ์ (ตามโค้ดเดิม)
trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values,    np.nan)
trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
trip_match["Match_Hotel_Ref"]     = np.where(is_match, matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip_match))).values, np.nan)

# สรุปภาพรวม
total   = len(trip_match)
matched = int(trip_match["is_match"].sum())
print(f"Matched {matched}/{total} rows ({matched/total:.1%}) with KEY_THRESH={KEY_THRESH}")
trip_match.head(20)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Matched 637/2665 rows (23.9%) with KEY_THRESH=0.95


,Trip_Name,Trip_URL,Trip_Address,OTA,Trip_Name_n,Trip_Address_n,Trip_Key,Match_Hotel_Name,Match_Hotel_Address,Match_Hotel_Ref,sim_name,sim_addr,sim_key,is_match,Best_Hotel_Name,Best_Hotel_Address,Best_Hotel_Ref
0,นาซ่า กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"44 ถ. สุขุมวิท 71 แขวงสวนหลวง, เขตสวนหลวง, กรุ...",trip_com,นาซ่า,44สุขุมวิท71แขวงสวนหลวงเขตสวนหลวงกรุงเทพมหานคร...,นาซ่า 44สุขุมวิท71แขวงสวนหลวงเขตสวนหลวงกรุงเทพ...,NaN,NaN,NaN,0.416311,0.768178,0.758598,False,โรงแรมเบสท์เวสเทิร์น นาดา ดอนเมือง แอร์พอร์ต,235/8 ถ.พหลโยธิน แขวงอนุสาวรีย์ เขตบางเขน จ.กร...,150/2565
1,โรงแรมอีสติน แกรนด์ พญาไท,https://th.trip.com/hotels/detail/?cityId=359&...,"18 ถ.พญาไท ทุ่งพญาไท, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,อีสตินแกรนด์พญาไท,18พญาไททุ่งพญาไทเขตราชเทวีกรุงเทพมหานคร10400,อีสตินแกรนด์พญาไท 18พญาไททุ่งพญาไทเขตราชเทวีกร...,โรงแรม อีสติน แกรนด์ พญาไท,18 ถ.พญาไท แขวงทุ่งพญาไท เขตราชเทวี จ.กรุงเทพม...,19/2566,1.000000,0.966716,0.990144,True,โรงแรม อีสติน แกรนด์ พญาไท,18 ถ.พญาไท แขวงทุ่งพญาไท เขตราชเทวี จ.กรุงเทพม...,19/2566
2,โรงแรมเอเชียกรุงเทพ,https://th.trip.com/hotels/detail/?cityId=359&...,"269 ถ.พ, เขตราชเทวี, กรุงเทพฯ, 10400",trip_com,เอเชีย,269พเขตราชเทวีกรุงเทพมหานคร10400,เอเชีย 269พเขตราชเทวีกรุงเทพมหานคร10400,NaN,NaN,NaN,1.000000,0.905918,0.923781,False,เอเชีย,296 ถ.พญาไท แขวงถนนเพชรบุรี เขตราชเทวี จ.กรุงเ...,107/2564
3,โรงแรมทวิน ทาวเวอร์,https://th.trip.com/hotels/detail/?cityId=359&...,"88 รองเมือง ปทุมวัน, ถ.พระราม 6, เขตปทุมวัน, ก...",trip_com,ทวินทาวเวอร์,88รองเมืองปทุมวันพระราม6เขตปทุมวันกรุงเทพมหานค...,ทวินทาวเวอร์ 88รองเมืองปทุมวันพระราม6เขตปทุมวั...,เดอะ ทวิน ทาวเวอร์ กรุงเทพฯ,88 ซ.รองเมือง 2 แขวงรองเมือง เขตปทุมวัน จ.กรุง...,74/2564,0.929925,0.972836,0.971434,True,เดอะ ทวิน ทาวเวอร์ กรุงเทพฯ,88 ซ.รองเมือง 2 แขวงรองเมือง เขตปทุมวัน จ.กรุง...,74/2564
4,โรงแรมเดอะ เบอร์เคลีย์ ประตูน้ำ,https://th.trip.com/hotels/detail/?cityId=359&...,"559 ราชปรารภ, มักกะสัน ราชเทวี, เขตราชเทวี, กร...",trip_com,เดอะเบอร์เคลีย์ประตูน้ํา,559ราชปรารภมักกะสันราชเทวีเขตราชเทวีกรุงเทพมหา...,เดอะเบอร์เคลีย์ประตูน้ํา 559ราชปรารภมักกะสันรา...,เดอะ เบอร์เคลีย์ โฮเต็ล ประตูน้ำ,559 ถ.ราชปรารภ แขวงมักกะสัน เขตราชเทวี จ.กรุงเ...,194/2566,0.966133,0.966027,0.971412,True,เดอะ เบอร์เคลีย์ โฮเต็ล ประตูน้ำ,559 ถ.ราชปรารภ แขวงมักกะสัน เขตราชเทวี จ.กรุงเ...,194/2566
5,12 เดอะ เรสซิเดนซ์ โฮเทลแอนด์อพาร์ตเมนต์,https://th.trip.com/hotels/detail/?cityId=359&...,"22 ซ.สรณคมน์ 12 แขวงสีกัน, เขตดอนเมือง, กรุงเท...",trip_com,12เดอะเรสซิเดนซ์โฮเทลแอนด์อพาร์ตเมนต์,22ซ.สรณคมน์12แขวงสีกันเขตดอนเมืองกรุงเทพมหานคร...,12เดอะเรสซิเดนซ์โฮเทลแอนด์อพาร์ตเมนต์ 22ซ.สรณค...,NaN,NaN,NaN,0.776499,0.958448,0.874566,False,โรงแรม เคที เกสต์เฮาส์,14 ซ.อินทามระ 44 แขวงรัชดาภิเษก เขตดินแดง จ.กร...,117/2565
6,ไอบิส สไตล์ กรุงเทพ สีลม,https://th.trip.com/hotels/detail/?cityId=359&...,"289 ถ.สีลม, เขตบางรัก, กรุงเทพฯ, 10500",trip_com,ไอบิสสไตล์สีลม,289สีลมเขตบางรักกรุงเทพมหานคร10500,ไอบิสสไตล์สีลม 289สีลมเขตบางรักกรุงเทพมหานคร10500,ไอบิส สไตล์ กรุงเทพ สีลม,289 ถ.สีลม แขวงสีลม เขตบางรัก จ.กรุงเทพมหานคร ...,162/2563,1.000000,0.975503,0.984055,True,ไอบิส สไตล์ กรุงเทพ สีลม,289 ถ.สีลม แขวงสีลม เขตบางรัก จ.กรุงเทพมหานคร ...,162/2563
7,AETAS ลุมพินี,https://th.trip.com/hotels/detail/?cityId=359&...,"1030 4 ถ. พระรามที่ ๔ แขวงท, เขตสาทร, กรุงเทพฯ...",trip_com,aetasลุมพินี,10304พระรามที่๔แขวงทเขตสาทรกรุงเทพมหานคร10120,aetasลุมพินี 10304พระรามที่๔แขวงทเขตสาทรกรุงเท...,NaN,NaN,NaN,0.760066,0.881806,0.914838,False,พระโขนง,58 ซ.สุขุมวิท 46 ถ.พระราม 4 แขวงพระโขนง เขตคลอ...,120/2564
8,เมอร์เคียว กรุงเทพ สุรวงศ์,https://th.trip.com/hotels/detail/?cityId=359&...,"222 ถนนสุรวงศ์ แขวงสี่พระยา, เขตบางรัก, กรุงเท...",trip_com,เมอร์เคียวสุรวงศ์,222ถนนสุรวงศ์แขวงสี่พระยาเขตบางรักกรุงเทพมหานค...,เมอร์เคียวสุรวงศ์ 222ถนนสุรวงศ์แขวงสี่พระยาเขต...,NaN,NaN,NaN,1.000000,0.867777,0.872151,False,โรงแรมเมอร์เคียว กรุงเทพ สุรวงศ์,222 ถ.สุรวงศ์ แขวงสี่พระยา เขตบางรัก จ.กรุงเทพ...,56/2567
9,ชาเทรียม แกรนด์ กรุงเทพฯ,https://th.trip.com/hotels/detail/?cityId=359&...,"728 ถ., เขตราชเทวี, กรุงเทพฯ, 10400",tr

In [ ]:
trip_match.is_match.value_counts()

## Method 2: Ollama

In [12]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [32]:
# start the Ollama server in the background (serves on 127.0.0.1:11434)
!nohup ollama serve > /tmp/ollama.log 2>&1 &
!sleep 2

In [33]:
# Should return JSON like: {"status":"ok"}
!curl -s http://127.0.0.1:11434/api/version


{"version":"0.12.5"}

In [34]:
# Pull the Thai/English instruct model
!ollama pull scb10x/typhoon2.1-gemma3-4b:latest


In [35]:
!ollama pull bge-m3

In [27]:
# ---------- ตั้งค่า ----------
import re, unicodedata, json, math
import numpy as np
import pandas as pd
import requests
from sklearn.neighbors import NearestNeighbors

# -------------------------
# ฟังก์ชันขอ embeddings จาก Ollama
# -------------------------
OLLAMA_URL = "http://localhost:11434"
EMB_MODEL = "bge-m3"   # multilingual เหมาะกับไทย/อังกฤษ

def ollama_embed(texts, model=EMB_MODEL):
    """
    เรียก Ollama /api/embeddings ทีละรายการ (API ปัจจุบันรับทีละสตริง)
    แล้วต่อกลับเป็นอาร์เรย์ NxD พร้อม normalize เป็น unit vectors
    """
    embs = []
    for t in texts:
        resp = requests.post(
            f"{OLLAMA_URL}/api/embeddings",
            json={"model": model, "input": t if isinstance(t, str) else str(t)},
            timeout=60
        )
        resp.raise_for_status()
        data = resp.json()
        v = np.array(data["embedding"], dtype=np.float32)
        # L2 normalize (ให้เทียบเท่า normalize_embeddings=True ของ sentence-transformers)
        n = np.linalg.norm(v)
        if n > 0:
            v = v / n
        embs.append(v)
    return np.vstack(embs)

# =========================
# 0) เตรียมตารางต้นทาง (คงโค้ดของคุณ)
# =========================
trip = OTA_df.rename(columns={
    "name": "Trip_Name",
    "name href": "Trip_URL",
    "address": "Trip_Address"
}).copy().head(20)

hotels = hotel.copy()
if "v-data-table__td 2" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 2": "Hotel_Name"})
if "v-data-table__td 4" in hotels.columns:
    hotels = hotels.rename(columns={"v-data-table__td 4": "Hotel_Address"})
if "v-data-table__td" in hotels.columns:
    hotels["Hotel_Ref"] = hotels["v-data-table__td"]

for c in ["Trip_Name","Trip_Address"]:
    if c not in trip.columns: trip[c] = ""
trip[["Trip_Name","Trip_Address"]] = trip[["Trip_Name","Trip_Address"]].fillna("")

for c in ["Hotel_Name","Hotel_Address"]:
    if c not in hotels.columns: hotels[c] = ""
hotels[["Hotel_Name","Hotel_Address"]] = hotels[["Hotel_Name","Hotel_Address"]].fillna("")

# =========================
# 1) Clean + คีย์
# =========================
def normalize_name(s: str) -> str:
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower()
    s = re.sub(r"(โรงแรม|hotel)", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"กรุงเทพ(?:มหานคร|ฯ)?", " ", s)
    s = re.sub(r"[,\|\(\)\[\]{}\"'’`•·/.:;#&\-–—_+*^@!$?~]", " ", s)
    s = re.sub(r"\s+", "", s)
    return s

def normalize_addr(s: str) -> str:
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKC", str(s)).lower().strip()
    rep = {
        "กทม.": "กรุงเทพมหานคร",
        "กรุงเทพฯ": "กรุงเทพมหานคร",
        "จ.": "",
        "เขต ": "",
        "แขวง ": "",
        "ถ.": "",
        "ถนน ": "",
        "บ้านเลขที่": "",
    }
    for k,v in rep.items():
        s = s.replace(k, v)
    s = re.sub(r"[\"'’`•·\|\[\]{}#@!$^*?_+=:;,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return "".join(s.split())

trip["Trip_Name_n"]    = trip["Trip_Name"].map(normalize_name)
trip["Trip_Address_n"] = trip["Trip_Address"].map(normalize_addr)
hotels["Hotel_Name_n"]    = hotels["Hotel_Name"].map(normalize_name)
hotels["Hotel_Address_n"] = hotels["Hotel_Address"].map(normalize_addr)

trip["Trip_Key"]    = (trip["Trip_Name_n"]    + " " + trip["Trip_Address_n"]).str.strip()
hotels["Hotel_Key"] = (hotels["Hotel_Name_n"] + " " + hotels["Hotel_Address_n"]).str.strip()

# =========================
# 2) สร้าง embeddings ผ่าน Ollama
# =========================
emb_trip_name  = ollama_embed(trip["Trip_Name_n"].tolist())
emb_trip_addr  = ollama_embed(trip["Trip_Address_n"].tolist())
emb_trip_key   = ollama_embed(trip["Trip_Key"].tolist())

emb_hotel_name = ollama_embed(hotels["Hotel_Name_n"].tolist())
emb_hotel_addr = ollama_embed(hotels["Hotel_Address_n"].tolist())
emb_hotel_key  = ollama_embed(hotels["Hotel_Key"].tolist())

# =========================
# 3) ค้นหาคู่ + คำนวณ sim_name/sim_addr
# =========================
nn = NearestNeighbors(n_neighbors=1, metric="cosine")
nn.fit(emb_hotel_key)
dist, idx = nn.kneighbors(emb_trip_key, return_distance=True)
sim_key  = 1.0 - dist.ravel()
best_idx = idx.ravel()

best_name = np.einsum("ij,ij->i", emb_trip_name, emb_hotel_name[best_idx])
best_addr = np.einsum("ij,ij->i", emb_trip_addr, emb_hotel_addr[best_idx])

KEY_THRESH = 0.95
is_match = (sim_key >= KEY_THRESH)

# =========================
# 4) สร้างผลลัพธ์เหมือนเดิม
# =========================
matched_hotels = hotels.iloc[best_idx].reset_index(drop=True)

trip_match = trip.copy()
trip_match["Match_Hotel_Name"]    = np.where(is_match, matched_hotels["Hotel_Name"].values,    np.nan)
trip_match["Match_Hotel_Address"] = np.where(is_match, matched_hotels["Hotel_Address"].values, np.nan)
trip_match["Match_Hotel_Ref"]     = np.where(is_match, matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip))).values, np.nan)

trip_match["sim_name"] = best_name
trip_match["sim_addr"] = best_addr
trip_match["sim_key"]  = sim_key
trip_match["is_match"] = is_match

trip_match["Best_Hotel_Name"]    = matched_hotels["Hotel_Name"].values
trip_match["Best_Hotel_Address"] = matched_hotels["Hotel_Address"].values
trip_match["Best_Hotel_Ref"]     = matched_hotels.get("Hotel_Ref", pd.Series([np.nan]*len(trip_match))).values

total   = len(trip_match)
matched = int(trip_match["is_match"].sum())
print(f"Matched {matched}/{total} rows ({matched/total:.1%}) with KEY_THRESH={KEY_THRESH}")
trip_match.head(20)


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78b0fda002f0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [36]:
import requests, json
import numpy as np

TY_MODEL = "scb10x/typhoon2.1-gemma3-4b:latest"
KEY_LOWER = 0.85        # เรียก LLM เฉพาะเคสก้ำกึ่ง
KEY_THRESH = 0.95       # ใช้ตัวเดียวกับที่คุณตั้งไว้

def build_prompt_row(a_name, a_addr, b_name, b_addr, sim_key, sim_name, sim_addr, key_thresh=KEY_THRESH):
    tmpl = """คุณคือผู้ช่วยตรวจจับการจับคู่โรงแรมจากสองแหล่งข้อมูลภาษาไทย/อังกฤษ
จงตัดสินว่า A และ B คือ "โรงแรมเดียวกัน" หรือ "คนละโรงแรม" โดยพิจารณาจากชื่อและที่อยู่เป็นหลัก

ข้อควรระวัง:
- ชื่อที่มีคำว่า "โรงแรม / hotel" ให้ถือว่าเป็นคำทั่ว ๆ ไป ไม่ใช่สาระสำคัญ
- ที่อยู่ต้องอ้างถึงย่าน/ถนน/แขวง/เขต/จังหวัดเดียวกันหรือสอดคล้องกันอย่างชัดเจน
- การสะกด/ช่องว่างเพี้ยนเล็กน้อยให้มองว่าเท่ากันได้ ถ้าความหมายเหมือนเดิม
- ถ้าชื่อเหมือนมากแต่ที่อยู่คนละเขต/จังหวัด ให้ถือว่า "คนละโรงแรม"
- ให้ใช้สกอร์ความคล้ายเป็นข้อมูลประกอบการตัดสินใจ ไม่ใช่ตัวชี้ขาด

ข้อมูล:
A.name = "{a_name}"
A.address = "{a_addr}"
B.name = "{b_name}"
B.address = "{b_addr}"

similarities:
sim_key  = {sim_key:.4f}
sim_name = {sim_name:.4f}
sim_addr = {sim_addr:.4f}

เกณฑ์แนะนำ:
- ถ้า sim_key >= {key_thresh:.2f} ให้เอนเอียงไปทาง "ใช่ (same)"
- ถ้า 0.85 <= sim_key < {key_thresh:.2f} ให้ตรวจละเอียดจากที่อยู่ (แขวง/เขต/จังหวัด/ถนน)
- ถ้า sim_key < 0.85 ส่วนใหญ่เป็น "ไม่ใช่ (different)" เว้นแต่ที่อยู่จะยืนยันชัดมาก

รูปแบบคำตอบ: ตอบเป็น JSON ก้อนเดียวเท่านั้น โดยไม่มีข้อความอื่น เช่น
{{
  "same_hotel": true,
  "confidence": 0.0,
  "reason": "สรุปเหตุผลสั้น ๆ เป็นภาษาไทย (<= 2 บรรทัด)"
}}

ห้ามคืนค่าอื่นนอกเหนือจาก JSON"""
    return tmpl.format(
        a_name=a_name or "",
        a_addr=a_addr or "",
        b_name=b_name or "",
        b_addr=b_addr or "",
        sim_key=sim_key if np.isfinite(sim_key) else 0.0,
        sim_name=sim_name if np.isfinite(sim_name) else 0.0,
        sim_addr=sim_addr if np.isfinite(sim_addr) else 0.0,
        key_thresh=key_thresh
    )

def llm_verify_one(i, timeout=60):
    """ตรวจแถวที่ i ด้วย Typhoon 4B; คืน dict หรือ None ถ้าพาร์สไม่ได้"""
    prompt = build_prompt_row(
        trip.loc[i,"Trip_Name"], trip.loc[i,"Trip_Address"],
        matched_hotels.loc[i,"Hotel_Name"], matched_hotels.loc[i,"Hotel_Address"],
        sim_key[i], best_name[i], best_addr[i], KEY_THRESH
    )
    r = requests.post(
        f"{OLLAMA_URL}/api/generate",
        json={
            "model": TY_MODEL,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0.0},
            # ถ้า Ollama รุ่นที่คุณใช้รองรับ `format: "json"` ให้ใส่บรรทัดด้านล่างนี้
            # "format": "json"
        },
        timeout=timeout
    )
    r.raise_for_status()
    text = r.json().get("response","").strip()
    # กันกรณีโมเดลพูดเกิน — ดึง JSON ส่วนแรก
    try:
        # กรณีตอบเป็น JSON ตรง ๆ
        data = json.loads(text)
    except json.JSONDecodeError:
        # พยายามสกัดบล็อค {...}
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1 and end > start:
            data = json.loads(text[start:end+1])
        else:
            return None
    return data

# สร้างคอลัมน์ผลจาก LLM
trip_match["llm_same"] = np.nan
trip_match["llm_conf"] = np.nan
trip_match["llm_reason"] = np.nan

# เรียก LLM เฉพาะเคสก้ำกึ่ง เพื่อประหยัดเวลา
mask_candidates = (sim_key >= KEY_LOWER) & (sim_key < KEY_THRESH)
idxs = np.where(mask_candidates)[0]

for i in idxs:
    out = llm_verify_one(i)
    if isinstance(out, dict) and "same_hotel" in out:
        trip_match.at[i, "llm_same"]   = bool(out.get("same_hotel"))
        trip_match.at[i, "llm_conf"]   = float(out.get("confidence", 0.0))
        trip_match.at[i, "llm_reason"] = str(out.get("reason", "")).strip()

# ถ้าต้องการ “ยกผล LLM มาทับ” การตัดสินใจเดิมสำหรับเคสก้ำกึ่ง:
is_match_refined = is_match.copy()
for i in idxs:
    v = trip_match.at[i, "llm_same"]
    if pd.notna(v):
        is_match_refined[i] = bool(v)

trip_match["is_match_refined"] = is_match_refined


ReadTimeout: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)